In [8]:
import os

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

api_key = os.environ['OPENAI_API_KEY']

model = ChatOpenAI(api_key=api_key, temperature=0)

# ---- Define output data types within the class and initialize a parser ----
class Players(BaseModel):
    values: list = Field(description="Python list of dictionaries containing player name and nathionality and a team name")
    country: str = Field(description="Give me the most popular country accross the results")

parser = PydanticOutputParser(pydantic_object=Players)

# print(parser.get_format_instructions())
# ------------------------- Setup the request -------------------------
human_promt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")
chat_prompt = ChatPromptTemplate.from_messages([human_promt]) # uses human_promt template to make request to a llm

request = chat_prompt.format_prompt(
    request="Give me facts about 100 NBA players around the world",
    format_instructions=parser.get_format_instructions()
).to_messages()

##  How do we want to take the results from LLM
# it's like predict = model(X_test)
results = model(request, temperature=0)

# Here we parse results from LLM
results_values = parser.parse(results.content)

print(f'results_values: {results_values}, type: {type(results_values)}')

# ------------------------- Setup the results -------------------------
import pandas as pd

results_df = pd.DataFrame(results_values.values)

print(results_df.head(5))
print(results_df.shape)

print(f'The most popular country is {results_values.country}')

results_values: values=[{'player_name': 'LeBron James', 'nationality': 'United States', 'team_name': 'Los Angeles Lakers'}, {'player_name': 'Giannis Antetokounmpo', 'nationality': 'Greece', 'team_name': 'Milwaukee Bucks'}, {'player_name': 'Luka Doncic', 'nationality': 'Slovenia', 'team_name': 'Dallas Mavericks'}, {'player_name': 'Kevin Durant', 'nationality': 'United States', 'team_name': 'Brooklyn Nets'}, {'player_name': 'Nikola Jokic', 'nationality': 'Serbia', 'team_name': 'Denver Nuggets'}, {'player_name': 'Stephen Curry', 'nationality': 'United States', 'team_name': 'Golden State Warriors'}, {'player_name': 'Joel Embiid', 'nationality': 'Cameroon', 'team_name': 'Philadelphia 76ers'}, {'player_name': 'Kawhi Leonard', 'nationality': 'United States', 'team_name': 'LA Clippers'}, {'player_name': 'Rudy Gobert', 'nationality': 'France', 'team_name': 'Utah Jazz'}, {'player_name': 'Jayson Tatum', 'nationality': 'United States', 'team_name': 'Boston Celtics'}] country='United States', type:

In [9]:
results_df.head()

,player_name,nationality,team_name
0,LeBron James,United States,Los Angeles Lakers
1,Giannis Antetokounmpo,Greece,Milwaukee Bucks
2,Luka Doncic,Slovenia,Dallas Mavericks
3,Kevin Durant,United States,Brooklyn Nets
4,Nikola Jokic,Serbia,Denver Nuggets


In [12]:
results_values.values

[{'player_name': 'LeBron James',
  'nationality': 'United States',
  'team_name': 'Los Angeles Lakers'},
 {'player_name': 'Giannis Antetokounmpo',
  'nationality': 'Greece',
  'team_name': 'Milwaukee Bucks'},
 {'player_name': 'Luka Doncic',
  'nationality': 'Slovenia',
  'team_name': 'Dallas Mavericks'},
 {'player_name': 'Kevin Durant',
  'nationality': 'United States',
  'team_name': 'Brooklyn Nets'},
 {'player_name': 'Nikola Jokic',
  'nationality': 'Serbia',
  'team_name': 'Denver Nuggets'},
 {'player_name': 'Stephen Curry',
  'nationality': 'United States',
  'team_name': 'Golden State Warriors'},
 {'player_name': 'Joel Embiid',
  'nationality': 'Cameroon',
  'team_name': 'Philadelphia 76ers'},
 {'player_name': 'Kawhi Leonard',
  'nationality': 'United States',
  'team_name': 'LA Clippers'},
 {'player_name': 'Rudy Gobert',
  'nationality': 'France',
  'team_name': 'Utah Jazz'},
 {'player_name': 'Jayson Tatum',
  'nationality': 'United States',
  'team_name': 'Boston Celtics'}]